In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
#
# !ln -s ./drive/MyDrive/Data/DogCatCls/save/ ./save
#
# %%shell
# cp ./drive/MyDrive/Data/DogCatCls/dog-vs-cat-classification.zip .
# unzip dog-vs-cat-classification.zip -d dataset > /dev/null
#
# !cp ./drive/MyDrive/Data/DogCatCls/train_labels.csv .
#
# !pip3 install timm tqdm


/home/thaiminhpv/anaconda3/envs/thaiminhpv/lib/python3.9/site-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


In [ ]:

# libraries
import os
import time
import subprocess
import numpy as np
import pandas as pd
import ast
import cv2
import PIL.Image
import matplotlib.pyplot as plt
import timm
%matplotlib inline
import seaborn as sns
import torch
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR
# from warmup_scheduler import GradualWarmupScheduler
import albumentations
import torch.cuda.amp as amp
# import segmentation_models_pytorch as smp
from tqdm import tqdm

scaler = amp.GradScaler()
# device = torch.device('cuda')

In [3]:
from torchvision.io import read_image

In [4]:
TRAIN_DIR = 'dataset/train/train/'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# dog is 1 and cat is 0

In [5]:
df = pd.read_csv('train_labels.csv')
df

,filepath,id,labels
0,dataset/train/train/dogs/dog.9265.jpg,dog.9265.jpg,1
1,dataset/train/train/dogs/dog.4445.jpg,dog.4445.jpg,1
2,dataset/train/train/dogs/dog.3578.jpg,dog.3578.jpg,1
3,dataset/train/train/dogs/dog.7244.jpg,dog.7244.jpg,1
4,dataset/train/train/dogs/dog.7907.jpg,dog.7907.jpg,1
...,...,...,...
24995,dataset/train/train/cats/cat.9419.jpg,cat.9419.jpg,0
24996,dataset/train/train/cats/cat.3532.jpg,cat.3532.jpg,0
24997,dataset/train/train/cats/cat.2772.jpg,cat.2772.jpg,0
24998,dataset/train/train/cats/cat.10915.jpg,cat.10915.jpg,0


In [6]:
df = df.drop(columns=['id']).sample(frac=1).reset_index(drop=True)
df

,filepath,labels
0,dataset/train/train/dogs/dog.10688.jpg,1
1,dataset/train/train/cats/cat.12340.jpg,0
2,dataset/train/train/dogs/dog.9315.jpg,1
3,dataset/train/train/cats/cat.996.jpg,0
4,dataset/train/train/dogs/dog.5656.jpg,1
...,...,...
24995,dataset/train/train/cats/cat.4943.jpg,0
24996,dataset/train/train/dogs/dog.9913.jpg,1
24997,dataset/train/train/dogs/dog.12117.jpg,1
24998,dataset/train/train/dogs/dog.1220.jpg,1


In [250]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [251]:
class DogCatCls(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        # image = read_image(self.df.iloc[idx]['filepath'])
        image = cv2.imread(self.df.iloc[idx]['filepath'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        image = np.transpose(image, (2, 0, 1))
        image = image.astype(np.float32)
        image /= 255.0
        if self.transform:
            image = self.transform(image=image)
        image = torch.from_numpy(image)
        label = torch.FloatTensor([df.iloc[idx]['labels'].item()])
        return image, label


In [252]:
import timm
# create model
class DogCatClsModel(nn.Module):
    def __init__(self, num_classes):
        super(DogCatClsModel, self).__init__()
        self.enet = timm.create_model('tf_efficientnet_b0_ns', pretrained=True)
        self.enet.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(self.enet.classifier.in_features, num_classes)
        )
        self.enet.classifier = nn.Identity()
        # for layer in self.enet.parameters():
        #     layer.requires_grad = False
    def forward(self, x):
        x = self.enet(x)
        return x

m = DogCatClsModel(num_classes=1)
m(torch.randn(1, 3, 224, 224))

tensor([[-0.1052]], grad_fn=<AddmmBackward0>)

In [265]:
criterion = nn.BCEWithLogitsLoss()
def train_epoch(model, loader, optimizer):
    model.train()
    train_loss = []
    bar = tqdm(loader)
    for images, labels in bar:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        with amp.autocast():
            logits = model(images)
            loss = criterion(logits, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loss_np = loss.item()
        train_loss.append(loss_np)
        bar.set_description('loss: %0.4f' % loss_np)
    return train_loss

def val_epoch(model, loader):
    model.eval()
    val_loss = []
    bar = tqdm(loader)
    with torch.no_grad():
        for images, labels in bar:
            images = images.to(device)
            labels = labels.to(device)
            logit = model(images)
            loss = criterion(logit, labels)
            loss_np = loss.item()
            val_loss.append(loss_np)
            bar.set_description('loss: %0.4f' % loss_np)
    return val_loss


In [257]:

def run(epoch_num):
    model = DogCatClsModel(num_classes=1)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
    # scaler = torch.cuda.amp.GradScaler()
    dataset_train = DataLoader(
        DogCatCls(train_df, transform=None),
        batch_size=32,
        shuffle=True,
        num_workers=4
    )

    dataset_val = DataLoader(
        DogCatCls(val_df),
        batch_size=32,
        shuffle=True,
        num_workers=4
    )
    for epoch in range(1, epoch_num):
        train_loss = train_epoch(model, dataset_train, optimizer)
        val_loss = val_epoch(model, dataset_val)
        scheduler.step(np.mean(val_loss), epoch)
        print(f'Epoch: {epoch}, Train Loss: {np.mean(train_loss)}, Val Loss: {np.mean(val_loss)}')
        # save model
        torch.save(model.state_dict(), f'save/model_epoch_{epoch}.pth')
        # save loss
        np.save(f'save/loss_epoch_{epoch}.npy', np.array(train_loss))
        np.save(f'save/val_loss_epoch_{epoch}.npy', np.array(val_loss))


In [255]:
run(30)

  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [295]:
TEST_DIR = 'dataset/test/test/'

df_test = pd.read_csv('./dataset/sample_submission.csv')
df_test['filepath'] = TEST_DIR + df_test['id']
df_test

,id,labels,filepath
0,000000.jpg,1,dataset/test/test000000.jpg
1,000001.jpg,1,dataset/test/test000001.jpg
2,000002.jpg,1,dataset/test/test000002.jpg
3,000003.jpg,1,dataset/test/test000003.jpg
4,000004.jpg,1,dataset/test/test000004.jpg
...,...,...,...
7995,007995.jpg,0,dataset/test/test007995.jpg
7996,007996.jpg,0,dataset/test/test007996.jpg
7997,007997.jpg,0,dataset/test/test007997.jpg
7998,007998.jpg,0,dataset/test/test007998.jpg


In [ ]:

def inference():
    model = DogCatClsModel(num_classes=1)
    model.load_state_dict(torch.load('save/model_epoch_30.pth'))
    model.to(device)
    dataset_test = DataLoader(
        DogCatCls(df_test, transform=None),
        batch_size=32,
        shuffle=False,
        num_workers=4
    )
    model.eval()
    preds = []
    with torch.no_grad():
        for images, labels in tqdm(dataset_test):
            images = images.to(device)
            logit = model(images)
            preds.append(logit.sigmoid().cpu().numpy())

    preds = np.concatenate(preds, axis=0)
    preds_ = (preds > 0.5).astype(np.int32)
    df_test['labels'] = preds_
    df_test.to_csv('save/submission.csv', index=False)